Notebook showing usage of the data detective.

In [1]:
# To import detective from relative folder
import detective.core as detective
import detective.functions as functions
import pandas as pd

To load the local db we supply the path to the sqlite `.db` file 

In [2]:
db = detective.HassDatabase('sqlite:///home-assistant_v2.db') # To init without fetching entities fetch_entities=False

Successfully connected to database sqlite:///home-assistant_v2.db
There are 10 entities with data


In [3]:
db.entities

['automation.new_file_alert',
 'camera.demo_camera',
 'camera.local_file',
 'group.all_automations',
 'history_graph.pms5003',
 'sensor.merton',
 'sensor.particles_01um',
 'sensor.particles_025um',
 'sensor.particles_10um',
 'sensor.serial_sensor']

Use `fetch_all_sensor_data()` to fetch all your sensor data into a pandas dataframe in memory. Note that by default the number of states returned is limited but this can optionally be over-ridden as shown below. 

In [4]:
df = db.fetch_all_sensor_data(limit=1e9)

The returned Pandas dataframe has 1545 rows of data.


Take a minute to read about the differences between [wide-form and long-form data](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data). The Pandas dataframe we have is in long form.

In [5]:
df.head()

,domain,entity_id,state,last_changed,attributes
0,sensor,sensor.particles_10um,154,2018-12-04 09:47:14.801458,"{""unit_of_measurement"": ""particles"", ""friendly..."
1,sensor,sensor.particles_025um,143,2018-12-04 09:47:14.801221,"{""unit_of_measurement"": ""particles"", ""friendly..."
2,sensor,sensor.particles_01um,107,2018-12-04 09:47:14.800889,"{""unit_of_measurement"": ""particles"", ""friendly..."
3,sensor,sensor.serial_sensor,"{ ""a"": 107, ""b"": 143, ""c"": 154 }",2018-12-04 09:47:14.799862,"{""a"": 107, ""b"": 143, ""c"": 154, ""friendly_name""..."
4,sensor,sensor.particles_10um,155,2018-12-04 09:47:13.888892,"{""unit_of_measurement"": ""particles"", ""friendly..."


It is necessary to do some formatting of the data before we can plot it, and detective provides several functions to assist. Additionally it will generate some features for you.

In [6]:
df = functions.generate_features(df)
df = functions.format_dataframe(df)

In [7]:
df.head()

,domain,entity_id,state,last_changed,attributes,device_class,unit_of_measurement,friendly_name
0,sensor,sensor.particles_10um,154.0,2018-12-04 09:47:14.801458,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um
1,sensor,sensor.particles_025um,143.0,2018-12-04 09:47:14.801221,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_025um
2,sensor,sensor.particles_01um,107.0,2018-12-04 09:47:14.800889,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_01um
4,sensor,sensor.particles_10um,155.0,2018-12-04 09:47:13.888892,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um
6,sensor,sensor.particles_10um,158.0,2018-12-04 09:47:12.978155,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um


Notice the new feature columns added. It is straightforward to create your own features, for example to add a `day_of_week` column

In [8]:
df['day_of_week'] = df['last_changed'].apply(lambda x : x.dayofweek)

In [9]:
df.head()

,domain,entity_id,state,last_changed,attributes,device_class,unit_of_measurement,friendly_name,day_of_week
0,sensor,sensor.particles_10um,154.0,2018-12-04 09:47:14.801458,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um,1
1,sensor,sensor.particles_025um,143.0,2018-12-04 09:47:14.801221,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_025um,1
2,sensor,sensor.particles_01um,107.0,2018-12-04 09:47:14.800889,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_01um,1
4,sensor,sensor.particles_10um,155.0,2018-12-04 09:47:13.888892,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um,1
6,sensor,sensor.particles_10um,158.0,2018-12-04 09:47:12.978155,"{'unit_of_measurement': 'particles', 'friendly...",unknown,particles,particles_10um,1


## Plot some data
I will use Altair

In [10]:
#!pip install altair # Uncomment to install altair

In [11]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

to_plot = 'particles_01um'
alt.Chart(df[df['friendly_name'] == to_plot]).mark_line().encode(
    x='last_changed',
    y='state',
    color='entity_id',
    tooltip=['entity_id', 'state', 'last_changed']
).properties(
    width=800,
    height=300
).interactive()

alt.Chart(...)